# Splitting data for one vs all technique 
#### NOTE:- dont re run this code!, seperating positive and negative data takes ~ 5 hrs to run

In [1]:
from scipy import sparse
import numpy as np
from numpy import savez_compressed
from numpy import load
from tqdm import tqdm
import os
import gc
from sklearn import utils
from scipy.sparse import csr_matrix

In [2]:
source = 'TF-IDF.npz'
path = r'TF_IDF'
technique = "TF_IDF"

In [17]:
source = 'BOW.npz'
path = r'BOW'
technique = "BOW"


In [3]:
count_text = sparse.load_npz(source)
print(type(count_text))
print(count_text.shape)

<class 'scipy.sparse._csr.csr_matrix'>
(395172, 3131007)


In [4]:
labels = load('labels.npy')
labels = np.array(labels)
print(type(labels))
print(labels.shape)

<class 'numpy.ndarray'>
(395172,)


## Splitting positive and negative Reviews

## Takes a ton of time to execute, need to find a better way to do this.

In [5]:
#original
n_samples = count_text.shape[0]

x_positive = csr_matrix((0, 122124), dtype = np.int8).toarray()
x_negative = csr_matrix((0, 122124), dtype = np.int8).toarray()

x_positive = []
x_negative = []

pc = 0
nc = 0
pos_count=0
for i in tqdm( range(n_samples) ):
    pos_count+=1

    if(labels[i] == 1):
        x_positive =  sparse.vstack((x_positive, count_text[i]))
    else:
        x_negative =  sparse.vstack((x_negative, count_text[i]))        

print(x_positive.shape)
print(x_negative.shape)

100%|██████████| 395172/395172 [22:37:06<00:00,  4.85it/s]         

(337810, 3131007)
(57364, 3131007)


In [6]:

n_pos = np.count_nonzero(labels)
n_neg = len(labels)-n_pos
print("number of positive reviews = ", n_pos)
print("number of positive reviews = ", n_neg)

print(n_pos, n_neg)

ratio = round(x_positive.shape[0]/x_negative.shape[0])
print("Ratio = ", ratio)
c = x_positive.shape[0]//ratio
print('count per split = ', c)

number of positive reviews =  337809
number of positive reviews =  57363
337809 57363
Ratio =  6
count per split =  56301


## Converting COO Matrix to CSR Matrix for slicing 

In [7]:
x_positive = csr_matrix(x_positive)
x_negative = csr_matrix(x_negative)

In [8]:
x_positive[i*c:c*(i+1)].shape

(0, 3131007)

In [9]:
c = x_positive.shape[0]//ratio
print('count per split = ', c)

x_sets = [] 
for i in range(ratio):
    #print(i*c, c*(i+1))
    #x_sets.append(sparse.vstack((x_positive[i*c:c*(i+1)], x_negative)))
    x_sets.append(x_positive[i*c:c*(i+1)])

count per split =  56301


sparse_matrix = scipy.sparse.load_npz('/tmp/sparse_matrix.npz')

array1, array2 = utils.shuffle(array1, array2)

## Storing the datasets

In [10]:
for i in range(len(x_sets)):
    f_path = os.path.join(path, technique+'_POS'+str(i)+'.npz')
    print(f_path,x_sets[i].shape)
    sparse.save_npz(f_path, x_sets[i])
    
f_path = os.path.join(path, technique+'_NEG'+'.npz')
print(f_path, x_negative.shape)
sparse.save_npz(f_path, x_negative)

TF_IDF\TF_IDF_POS0.npz (56301, 3131007)
TF_IDF\TF_IDF_POS1.npz (56301, 3131007)
TF_IDF\TF_IDF_POS2.npz (56301, 3131007)
TF_IDF\TF_IDF_POS3.npz (56301, 3131007)
TF_IDF\TF_IDF_POS4.npz (56301, 3131007)
TF_IDF\TF_IDF_POS5.npz (56301, 3131007)
TF_IDF\TF_IDF_NEG.npz (57364, 3131007)
